In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

# Machine Learning

## Scaling

In [1]:
# Define your scaling function
def custom_scaling(input_values):
    input_values_scaled = np.zeros_like(input_values, dtype=float)  # Initialize scaled data array
    input_values_scaled[:, 0] = (input_values[:, 0] - L_min) / (L_max - L_min)  # Scale L
    input_values_scaled[:, 1] = (input_values[:, 1] - C_min) / (C_max - C_min)  # Scale C
    input_values_scaled[:, 2] = (input_values[:, 2] - fsw_min) / (fsw_max - fsw_min)  # Scale fsw
    return input_values_scaled

## Learning rate + regularization

In [3]:
# Load your simulation data
# Replace "your_dataset.csv" with the path to your dataset
df = pd.read_csv(r"E:\ai-power-converter\simulation_results_new.csv")

# Define custom scaling ranges for each input feature
L_min, L_max = 30e-6, 2000e-6
C_min, C_max = 20e-6, 1000e-6
fsw_min, fsw_max = 20e3, 200e3


# Extract input features (L, C, fsw)
X = df[['L', 'C', 'fsw']].values

# Extract output (ripples)
y = df[["delta_current", "delta_voltage", "Pl_s1", "Pl_s2", "Pl_C", "Pl_L_Cu"]].values  # Adjust column names as per your dataset

# # Scale input features to range [0, 1]
# scaler_X = MinMaxScaler()
# scaler_X.fit(X)
# X_scaled = scaler_X.transform(X)

# Apply custom scaling
X_scaled = custom_scaling(X)

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, mode='auto', restore_best_weights=True)

# # Save the scaler for input features
# with open('scaler_X_1.pkl', 'wb') as f:
#     pickle.dump(scaler_X, f)

# Split the dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define a custom callback to print custom information at the end of each epoch
class PrintEpochInfo(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}/{self.params['epochs']}, Loss: {logs['loss']}, Val Loss: {logs['val_loss']}")


model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=(3,)),  # Scalarization layer
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Hidden layer 1 with L2 regularization
    tf.keras.layers.BatchNormalization(),                  # Batch normalization
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Hidden layer 2 with L2 regularization
    tf.keras.layers.BatchNormalization(),                  # Batch normalization
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Hidden layer 3 with L2 regularization
    tf.keras.layers.BatchNormalization(),         
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Hidden layer 2 with L2 regularization
    tf.keras.layers.BatchNormalization(),                  # Batch normalization
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Hidden layer 3 with L2 regularization
    tf.keras.layers.BatchNormalization(),       
    tf.keras.layers.Dense(6, activation='softplus')         # Output layer with 6 neurons
])


# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')



# Define the learning rate scheduler
lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3, min_lr=1e-6)

# Train the model with custom callback to print information at the end of each epoch and early stopping
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), 
                    callbacks=[PrintEpochInfo(), lr_scheduler, early_stopping], verbose=1)


# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


Epoch 1/100
591/591 [==============================] - 19s 23ms/step - loss: 1.1117 - val_loss: 0.6522 - lr: 0.0010
Epoch 2/100
591/591 [==============================] - 13s 22ms/step - loss: 0.4600 - val_loss: 0.3036 - lr: 0.0010
Epoch 3/100
591/591 [==============================] - 13s 23ms/step - loss: 0.3046 - val_loss: 0.3074 - lr: 0.0010
Epoch 4/100
591/591 [==============================] - 10s 17ms/step - loss: 0.2807 - val_loss: 0.2253 - lr: 0.0010
Epoch 5/100
591/591 [==============================] - 14s 23ms/step - loss: 0.2692 - val_loss: 0.2036 - lr: 0.0010
Epoch 6/100
591/591 [==============================] - 15s 25ms/step - loss: 0.2611 - val_loss: 0.2607 - lr: 0.0010
Epoch 7/100
591/591 [==============================] - 10s 18ms/step - loss: 0.2564 - val_loss: 0.2160 - lr: 0.0010
Epoch 8/100
255/591 [===========>..................] - ETA: 5s - loss: 0.2668

KeyboardInterrupt: 

In [39]:
# Optionally, save the model
model.save("model_5_256_80k.h5")

d:\Krishna\Python_3_92\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Evaluate the model on the testing set
mse = model.evaluate(X_test, y_test)
print("Mean Squared Error on Testing Set:", mse)

# Use the trained model to make predictions on the testing set
predictions = model.predict(X_test)

# Print the first few predictions
print("Predictions:")
print(predictions[:5])

# Print the corresponding true values
print("True Values:")
print(y_test[:5])

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error (MAE): {mae}')

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Calculate R-squared (R2) Score
r2 = r2_score(y_test, predictions)
print(f'R-squared (R2) Score: {r2}')

# Calculate Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100
print(f'Mean Absolute Percentage Error (MAPE): {mape}%')

375/375 [==============================] - 0s 710us/step - loss: 0.0555
Mean Squared Error on Testing Set: 0.05548654496669769
375/375 [==============================] - 0s 675us/step
Predictions:
[[0.46383455 4.3869815  3.3973832  3.395957   0.00987571 0.46012503]
 [1.3740784  3.5382082  2.2547264  2.2551157  0.0282976  0.54669344]
 [0.4021424  4.0381107  3.697013   3.6964185  0.01675989 0.4676913 ]
 [0.7982075  4.343183   2.4132886  2.4129455  0.07227656 0.48374102]
 [0.23240648 1.9963129  2.66569    2.6661773  0.04521699 0.9656302 ]]
True Values:
[[0.41726057 4.23320884 3.56941854 3.56941854 0.00989122 0.4860326 ]
 [1.38188949 3.49085965 2.26846064 2.26846064 0.02568494 0.5123448 ]
 [0.38840219 4.20084617 3.75758414 3.75758414 0.01696425 0.45229604]
 [0.80562    4.37668499 2.36678473 2.36678473 0.07388625 0.46656346]
 [0.23768022 1.86408166 2.62867624 2.62867624 0.04867354 0.96624371]]
Mean Absolute Error (MAE): 0.051189703461745444
Root Mean Squared Error (RMSE): 0.1229984750887734

In [6]:
print(X_test)

[[0.84239734 0.59292035 0.52631111]
 [0.16648169 0.7699115  0.94736667]
 [0.84239734 0.59292035 0.94736667]
 ...
 [0.2336293  0.38827434 0.42105   ]
 [0.62930078 0.68141593 0.26315556]
 [0.54439512 0.04756637 0.52631111]]


# Model

## Scaling

In [ ]:
def scale_input_values(input_values):

    input_values_scaled = np.zeros_like(input_values, dtype=float)  # Initialize scaled data array

    # Scale L
    input_values_scaled[0] = (input_values[0] - L_min) / (L_max - L_min)

    # Scale C
    input_values_scaled[1] = (input_values[1] - C_min) / (C_max - C_min)

    # Scale fsw
    input_values_scaled[2] = (input_values[2] - fsw_min) / (fsw_max - fsw_min)
    # Make predictions using the model

    # Reshape the input data to match the expected shape of the model
    input_values_reshaped = input_values_scaled.reshape((1, -1))

    return input_values_reshaped

## Load Model

In [46]:


# Load the model
model = tf.keras.models.load_model(r'D:\Krishna\ai-power-converter\machine-learning\model_5_256_80k.h5')


# Define the input values
input_values = np.array([0.000408,0.000125,29473])  # Replace these values with your actual input values

input_values_scaled = scale_input_values(input_values)

# Make predictions using the model
prediction = model.predict(input_values_scaled)
# 3e-05,3e-05,29473.684210526317,4.173315066606831,19.44152069744096,1.9789759711750268,1.9789759711750268,0.5375860903195137,1.1010299411829958
# 0.0006521052631578,0.0001194736842105,143157.89473684208,0.0867577616798147,1.8022857528367369,3.1017004916475472,3.1017004916475472,0.0155677936159264,1.3457323804555696

# Define the original output values obtained from your simulation
original_output = np.array([0.3932206509319096,2.277082204631459,1.9236226560385399,1.9236226560385399,0.07285358695253268,0.9681316909747676])  # Replace these values with your actual output values

# Calculate the mean absolute error between original and predicted output values
mae = np.mean(np.abs(original_output - prediction))

# Calculate Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((original_output - prediction) / original_output)) * 100

# Print the original and predicted output values for comparison
print("Original output:", original_output)
print("Predicted output:", prediction)
print("Mean Absolute Error:", mae)
print(f'Mean Absolute Percentage Error (MAPE): {mape}%')


1/1 [==============================] - 0s 87ms/step
Original output: [0.39322065 2.2770822  1.92362266 1.92362266 0.07285359 0.96813169]
Predicted output: [[2.4493676e-08 6.8260856e-02 7.3280740e-01 4.0203422e-01 1.6395954e-02
  1.1412088e+01]]
Mean Absolute Error: 2.63581000032992
Mean Absolute Percentage Error (MAPE): 249.04604565176456%


model_5_256.h5
Original output: [0.08675776 1.80228575 3.10170049 3.10170049 0.01556779 1.34573238]
Predicted output: [[0.07611273 2.215468   3.096919   3.093316   0.01535417 1.3317415 ]]
Mean Absolute Error: 0.07519960021255918
Mean Absolute Percentage Error (MAPE): 6.338601044481275%

model_3_256.h5
Original output: [0.08675776 1.80228575 3.10170049 3.10170049 0.01556779 1.34573238]
Predicted output: [[0.11215301 2.2457957  3.0929043  3.0988061  0.01894178 1.3158761 ]]
Mean Absolute Error: 0.08563766249885481
Mean Absolute Percentage Error (MAPE): 13.02466772681769%